In [2]:
import pandas as pd
import os
import subprocess
import glob

In [3]:
def add_all_line(df, line):
    all_strings = ''
    for strings in df.iloc[line]:
        if type(strings) is str:
            all_strings += strings + '     '
    return all_strings


# Resolved wo sys

In [4]:
cat_name = ['cat34lowpurity', 'cat34highpurity']
# mass_name=['MX250_MH125','MX500_MH125','MX1000_MH125']
mass_name=['MX500_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        print(mass,cat)
        mass_dir =mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkmodel/2017/"+mass_dir+"/Datacard_"+mass+"_2017_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkmodel/2017/"+mass_dir+"/Datacard_"+mass+"_2017_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkmodel/2017/"+mass_dir+"/Datacard_"+mass+"_2017_zzgg_"+cat+".txt"
        FHSLws_path="/afs/cern.ch/user/s/shsong/WWggDNN/bdt/PBDT_HH_FHSL_combine_2017/flashgginput/50epochcombine/"+mass+"/ws_gghh_"+mass+"_2017_combineFHSL_"+cat
        bbggws_path="/afs/cern.ch/user/s/shsong/WWggDNN/bdt/PBDT_HH_FHSL_combine_2017/flashgginput/50epochcombine/"+mass+"/ws_gghh_"+mass+"_2017_bbgg_"+cat
        zzggws_path="/afs/cern.ch/user/s/shsong/WWggDNN/bdt/PBDT_HH_FHSL_combine_2017/flashgginput/50epochcombine/"+mass+"/ws_gghh_"+mass+"_2017_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        # print(FHSLws_path)
        # print(bbggws_path)
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkmodel/2017/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)

        if FHSLsignalmodel and not bbggsignalmodel and not zzggsignalmodel:
            print("FHSL",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
         
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            print("FHSL, bbgg, zzgg ",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
               
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

MX500_MH125 cat34lowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkmodel/2017/MX500_MH125/Datacard_MX500_MH125_cat34lowpurity.txt
FHSL, bbgg, zzgg  MX500_MH125 cat34lowpurity
MX500_MH125 cat34highpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkmodel/2017/MX500_MH125/Datacard_MX500_MH125_cat34highpurity.txt
FHSL and zzgg MX500_MH125 cat34highpurity


# Boosted wo sys

In [ ]:
cat_name = ['cat12highpurity','cat12lowpurity']
mass_name=['MX1000_MH125','MX500_MH125','MX550_MH125','MX600_MH125','MX650_MH125','MX700_MH125','MX750_MH125','MX800_MH125','MX850_MH125', 'MX1100_MH125', 'MX1200_MH125', 'MX1300_MH125','MX1400_MH125','MX1500_MH125', 'MX1600_MH125', 'MX1700_MH125','MX1800_MH125','MX1900_MH125', 'MX2000_MH125', 'MX2200_MH125', 'MX2600_MH125', 'MX2800_MH125','MX2400_MH125','MX3000_MH125']
# mass_name=['MX1000_MH125','MX500_MH125','MX3000_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        mass_dir = "datacard_" + mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/"+mass+"/Datacard_"+mass+"_2017_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/"+mass+"/Datacard_"+mass+"_2017_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/"+mass+"/Datacard_"+mass+"_2017_zzgg_"+cat+".txt"
        FHSLws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_combineFHSL_"+cat
        bbggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_bbgg_"+cat
        zzggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        print(FHSLws_path)
        print(bbggws_path)
        if not os.path.exists(mass):
            os.makedirs(mass)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/" + mass + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)
        print(FHSLsignalmodel)
        print(bbggsignalmodel)
        print(zzggsignalmodel)
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
                # lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+ "\n")
                # lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+ "\n")
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            print("FHSL, bbgg, zzgg ",mass,cat)
            print(FHSLcat_path)
            print(bbggcat_path)
            print(zzggcat_path)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                # lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+"     "+df_zzgg.iloc[20][2]+ "\n")
                # lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+"     "+df_zzgg.iloc[21][2]+ "\n")
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
                # lines[24]=(add_all_line(df_FHSL,20) + df_zzgg.iloc[20][2]+ "\n")
                # lines[25]=(add_all_line(df_FHSL,21) + df_zzgg.iloc[21][2]+ "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_combineFHSL_cat12highpurity
/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_bbgg_cat12highpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/MX1000_MH125/Datacard_MX1000_MH125_cat12highpurity.txt
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_combineFHSL_cat12highpurity/CMS-HGG_sigfit_packaged_combineFHSL_cat12highpurity_2017.root']
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_bbgg_cat12highpurity/CMS-HGG_sigfit_packaged_bbgg_cat12highpurity_2017.root']
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_zzgg_cat12highpurity/CMS-HGG_sigfit_packaged_zzgg_cat12highpurity_2017.root']
FHSL, bbgg, zzgg  MX1000_MH125 cat12highpurity
/afs/cern.ch/u

In [4]:
cat_name = ['cat12highpurity','cat12lowpurity']
mass_name=['MX1000_MH125','MX500_MH125','MX550_MH125','MX600_MH125','MX650_MH125','MX700_MH125','MX750_MH125','MX800_MH125','MX850_MH125', 'MX1100_MH125', 'MX1200_MH125', 'MX1300_MH125','MX1400_MH125','MX1500_MH125', 'MX1600_MH125', 'MX1700_MH125','MX1800_MH125','MX1900_MH125', 'MX2000_MH125', 'MX2200_MH125', 'MX2600_MH125', 'MX2800_MH125','MX2400_MH125','MX3000_MH125']
# mass_name=['MX1000_MH125','MX500_MH125','MX3000_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        mass_dir = "datacard_" + mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/"+mass+"/Datacard_"+mass+"_2017_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/"+mass+"/Datacard_"+mass+"_2017_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/"+mass+"/Datacard_"+mass+"_2017_zzgg_"+cat+".txt"
        FHSLws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_combineFHSL_"+cat
        bbggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_bbgg_"+cat
        zzggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        print(FHSLws_path)
        print(bbggws_path)
        if not os.path.exists(mass):
            os.makedirs(mass)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/" + mass + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)
        print(FHSLsignalmodel)
        print(bbggsignalmodel)
        print(zzggsignalmodel)
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
                # lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+ "\n")
                # lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+ "\n")
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            print("FHSL, bbgg, zzgg ",mass,cat)
            print(FHSLcat_path)
            print(bbggcat_path)
            print(zzggcat_path)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                # lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+"     "+df_zzgg.iloc[20][2]+ "\n")
                # lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+"     "+df_zzgg.iloc[21][2]+ "\n")
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
                # lines[24]=(add_all_line(df_FHSL,20) + df_zzgg.iloc[20][2]+ "\n")
                # lines[25]=(add_all_line(df_FHSL,21) + df_zzgg.iloc[21][2]+ "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_combineFHSL_cat12highpurity
/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_bbgg_cat12highpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/checkboosted/2017datacard/MX1000_MH125/Datacard_MX1000_MH125_cat12highpurity.txt
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_combineFHSL_cat12highpurity/CMS-HGG_sigfit_packaged_combineFHSL_cat12highpurity_2017.root']
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_bbgg_cat12highpurity/CMS-HGG_sigfit_packaged_bbgg_cat12highpurity_2017.root']
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_zzgg_cat12highpurity/CMS-HGG_sigfit_packaged_zzgg_cat12highpurity_2017.root']
FHSL, bbgg, zzgg  MX1000_MH125 cat12highpurity
/afs/cern.ch/u

# resolved 13Sep

In [4]:
cat_name = ['cat34FHlowpurity', 'cat34FHhighpurity']
mass_name=['MX250_MH125', 'MX260_MH125','MX280_MH125', 'MX300_MH125', 'MX320_MH125', 'MX350_MH125', 'MX400_MH125', 'MX450_MH125', 'MX500_MH125', 'MX550_MH125', 'MX600_MH125', 'MX650_MH125', 'MX700_MH125', 'MX750_MH125', 'MX800_MH125', 'MX850_MH125','MX1000_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        print(mass,cat)
        mass_dir = "datacard_" + mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/"+mass_dir+"/Datacard_"+mass+"_2017_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/"+mass_dir+"/Datacard_"+mass+"_2017_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/"+mass_dir+"/Datacard_"+mass+"_2017_zzgg_"+cat+".txt"
        FHSLws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2017/"+mass+"/ws_gghh_"+mass+"_2017_combineFHSL_"+cat
        bbggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2017/"+mass+"/ws_gghh_"+mass+"_2017_bbgg_"+cat
        zzggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2017/"+mass+"/ws_gghh_"+mass+"_2017_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root')) 
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        FHSLsignaltxt = os.path.getsize(FHSLcat_path) > 1000
        bbggsignaltxt = os.path.getsize(bbggcat_path) > 1000
        zzggsignaltxt = os.path.getsize(zzggcat_path) > 1000
        if len(FHSLsignalmodel)>0:
            if FHSLsignaltxt:
                FHSLsignalmodel = True
            else:
                FHSLsignalmodel = False
        else:
            FHSLsignalmodel = False
        if len(bbggsignalmodel)>0:
            if bbggsignaltxt:
                bbggsignalmodel = True
            else:
                bbggsignalmodel = False
        else:
            bbggsignalmodel = False
        if len(zzggsignalmodel)>0:
            if zzggsignaltxt:
                zzggsignalmodel = True
            else:
                zzggsignalmodel = False
        else:
            zzggsignalmodel = False
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)
        # print(FHSLsignalmodel)
        # print(bbggsignalmodel)
        # print(zzggsignalmodel)
        if FHSLsignalmodel and not bbggsignalmodel and not zzggsignalmodel:
            print("FHSL",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            # df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            # df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+ "\n")
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            print("FHSL, bbgg, zzgg ",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+"     "+df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+"     "+df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+"     "+df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+"     "+df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+"     "+df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+"     "+df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+"     "+df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+"     "+df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+"     "+df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+"     "+df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+"     "+df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+"     "+df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+"     "+df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+"     "+df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+"     "+df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+"     "+df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+"     "+df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+"     "+df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+"     "+df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+"     "+df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+"     "+df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+"     "+df_zzgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+"     "+df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+"     "+df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+"     "+df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+"     "+df_zzgg.iloc[45][2]+ "\n")
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            # df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            # df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_zzgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_zzgg.iloc[45][2]+ "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

MX250_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_MX250_MH125/Datacard_MX250_MH125_cat34FHlowpurity.txt
FHSL, bbgg, zzgg  MX250_MH125 cat34FHlowpurity
MX260_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_MX260_MH125/Datacard_MX260_MH125_cat34FHlowpurity.txt
FHSL, bbgg, zzgg  MX260_MH125 cat34FHlowpurity
MX280_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_MX280_MH125/Datacard_MX280_MH125_cat34FHlowpurity.txt
FHSL, bbgg, zzgg  MX280_MH125 cat34FHlowpurity
MX300_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_MX300_MH125/Datacard_MX300_MH125_cat34FHlowpurity.txt
FHSL, bbgg, zzgg  MX300_MH125 cat34FHlowpurity
MX320_MH125 cat34FHlowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/

In [6]:
cat_name = ['cat34FHlowpurity', 'cat34FHhighpurity']
mass_name=['MX250_MH125', 'MX260_MH125','MX280_MH125', 'MX300_MH125', 'MX320_MH125', 'MX350_MH125', 'MX400_MH125', 'MX450_MH125', 'MX500_MH125', 'MX550_MH125', 'MX600_MH125', 'MX650_MH125', 'MX700_MH125', 'MX750_MH125', 'MX800_MH125', 'MX850_MH125','MX1000_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        print(mass,cat)
        mass_dir = "datacard_" + mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_zzgg_"+cat+".txt"
        FHSLws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2018/"+mass+"/ws_gghh_"+mass+"_2018_combineFHSL_"+cat
        bbggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2018/"+mass+"/ws_gghh_"+mass+"_2018_bbgg_"+cat
        zzggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat34/2018/"+mass+"/ws_gghh_"+mass+"_2018_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root')) 
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        FHSLsignaltxt = os.path.getsize(FHSLcat_path) > 1000
        bbggsignaltxt = os.path.getsize(bbggcat_path) > 1000
        zzggsignaltxt = os.path.getsize(zzggcat_path) > 1000
        if len(FHSLsignalmodel)>0:
            if FHSLsignaltxt:
                FHSLsignalmodel = True
            else:
                FHSLsignalmodel = False
        else:
            FHSLsignalmodel = False
        if len(bbggsignalmodel)>0:
            if bbggsignaltxt:
                bbggsignalmodel = True
            else:
                bbggsignalmodel = False
        else:
            bbggsignalmodel = False
        if len(zzggsignalmodel)>0:
            if zzggsignaltxt:
                zzggsignalmodel = True
            else:
                zzggsignalmodel = False
        else:
            zzggsignalmodel = False
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)
        # print(FHSLsignalmodel)
        # print(bbggsignalmodel)
        # print(zzggsignalmodel)
        if FHSLsignalmodel and not bbggsignalmodel and not zzggsignalmodel:
            print("FHSL",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            # df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            # df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+ "\n")
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            print("FHSL, bbgg, zzgg ",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+"     "+df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+"     "+df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+"     "+df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+"     "+df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+"     "+df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+"     "+df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+"     "+df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+"     "+df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+"     "+df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+"     "+df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+"     "+df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+"     "+df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+"     "+df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+"     "+df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+"     "+df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+"     "+df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+"     "+df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+"     "+df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+"     "+df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+"     "+df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+"     "+df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+"     "+df_zzgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+"     "+df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+"     "+df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+"     "+df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+"     "+df_zzgg.iloc[45][2]+ "\n")
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            # df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            # df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_zzgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_zzgg.iloc[45][2]+ "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

MX250_MH125 cat34lowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_MX250_MH125/Datacard_MX250_MH125_cat34lowpurity.txt
FHSL, bbgg, zzgg  MX250_MH125 cat34lowpurity
MX600_MH125 cat34lowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_MX600_MH125/Datacard_MX600_MH125_cat34lowpurity.txt
FHSL MX600_MH125 cat34lowpurity
MX1000_MH125 cat34lowpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_MX1000_MH125/Datacard_MX1000_MH125_cat34lowpurity.txt
FHSL MX1000_MH125 cat34lowpurity
MX250_MH125 cat34highpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_MX250_MH125/Datacard_MX250_MH125_cat34highpurity.txt
FHSL MX250_MH125 cat34highpurity
MX600_MH125 cat34highpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_MX600_MH125/Datacard_MX600_M

# boosted

In [6]:
cat_name = ['cat12highpurity','cat12lowpurity']
mass_name=['MX1000_MH125','MX500_MH125','MX550_MH125','MX600_MH125','MX650_MH125','MX700_MH125','MX750_MH125','MX800_MH125','MX850_MH125', 'MX1100_MH125', 'MX1200_MH125', 'MX1300_MH125','MX1400_MH125','MX1500_MH125', 'MX1600_MH125', 'MX1700_MH125','MX1800_MH125','MX1900_MH125', 'MX2000_MH125', 'MX2200_MH125', 'MX2600_MH125', 'MX2800_MH125','MX2400_MH125','MX3000_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        mass_dir = "datacard_" + mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/"+mass_dir+"/Datacard_"+mass+"_2017_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/"+mass_dir+"/Datacard_"+mass+"_2017_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/"+mass_dir+"/Datacard_"+mass+"_2017_zzgg_"+cat+".txt"
        FHSLws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_combineFHSL_"+cat
        bbggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_bbgg_"+cat
        zzggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/"+mass+"/ws_gghh_"+mass+"_2017_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        print(FHSLws_path)
        print(bbggws_path)
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)
        print(FHSLsignalmodel)
        print(bbggsignalmodel)
        print(zzggsignalmodel)
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+ "\n")
                # lines[50]=(add_all_line(df_FHSL,46) + df_bbgg.iloc[46][2]+ "\n")
                # lines[51]=(add_all_line(df_FHSL,47) + df_bbgg.iloc[47][2]+ "\n")
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'
            print(FHSLcat_path)
            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+"     "+df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+"     "+df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+"     "+df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+"     "+df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+"     "+df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+"     "+df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+"     "+df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+"     "+df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+"     "+df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+"     "+df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+"     "+df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+"     "+df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+"     "+df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+"     "+df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+"     "+df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+"     "+df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+"     "+df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+"     "+df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+"     "+df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+"     "+df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+"     "+df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+"     "+df_zzgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+"     "+df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+"     "+df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+"     "+df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+"     "+df_zzgg.iloc[45][2]+ "\n")
                # lines[50]=(add_all_line(df_FHSL,46) + df_bbgg.iloc[46][2]+"     "+df_zzgg.iloc[46][2]+ "\n")
                # lines[51]=(add_all_line(df_FHSL,47) + df_bbgg.iloc[47][2]+"     "+df_zzgg.iloc[47][2]+ "\n")
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2017_hgg", "gghhWWgg_2017_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2017_hgg', 'gghhWWgg_2017_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2017_hgg', 'gghhbbgg_2017_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2017_hgg', 'gghhzzgg_2017_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2017 "," combineFHSL_"+cat+"_2017 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2017","combineFHSL_"+cat+"_2017")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_zzgg.iloc[41][2]+ "\n")
                # lines[46]=(add_all_line(df_FHSL,42) + df_zzgg.iloc[42][2]+ "\n")
                lines[47]=(add_all_line(df_FHSL,43) + df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_zzgg.iloc[44][2]+ "\n")
                # lines[49]=(add_all_line(df_FHSL,45) + df_zzgg.iloc[45][2]+ "\n")
                # lines[50]=(add_all_line(df_FHSL,46) + df_zzgg.iloc[46][2]+ "\n")
                # lines[51]=(add_all_line(df_FHSL,47) + df_zzgg.iloc[47][2]+ "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_combineFHSL_cat12highpurity
/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_bbgg_cat12highpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_MX1000_MH125/Datacard_MX1000_MH125_cat12highpurity.txt
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_combineFHSL_cat12highpurity/CMS-HGG_sigfit_packaged_combineFHSL_cat12highpurity_2017.root']
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_bbgg_cat12highpurity/CMS-HGG_sigfit_packaged_bbgg_cat12highpurity_2017.root']
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2017/MX1000_MH125/ws_gghh_MX1000_MH125_2017_zzgg_cat12highpurity/CMS-HGG_sigfit_packaged_zzgg_cat12highpurity_2017.root']
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Data

TypeError: can only concatenate str (not "NoneType") to str

In [9]:
df_FHSL

,0,1,2,3,4,5,6
0,CMS,HGG,Datacard,-,2017,-,13TeV
1,Auto-generated,by,flashggFinalFits/Datacard/makeDatacard.py,None,None,None,None
2,Run,with:,combine,None,None,None,None
3,---------------------------------------------,None,None,None,None,None,None
4,imax,*,None,None,None,None,None
5,jmax,*,None,None,None,None,None
6,kmax,*,None,None,None,None,None
7,---------------------------------------------,None,None,None,None,None,None
8,shapes,gghhWWgg_2017_hgg,combineFHSL_cat12highpurity_2017,/eos/cms/store/group/phys_b2g/shsong/flashggws...,wsig_13TeV:hggpdfsmrel_gghh_2017_combineFHSL_c...,None,None
9,shapes,bkg_mass,combineFHSL_cat12highpurity_2017,/eos/cms/store/group/phys_b2g/shsong/flashggws...,multipdf:CMS_hgg_combineFHSL_cat12highpurity_2...,None,None


In [8]:
df_bbgg

,0,1,2,3,4,5,6
0,CMS,HGG,Datacard,-,2017,-,13TeV
1,Auto-generated,by,flashggFinalFits/Datacard/makeDatacard.py,None,None,None,None
2,Run,with:,combine,None,None,None,None
3,---------------------------------------------,None,None,None,None,None,None
4,imax,*,None,None,None,None,None
5,jmax,*,None,None,None,None,None
6,kmax,*,None,None,None,None,None
7,---------------------------------------------,None,None,None,None,None,None
8,shapes,gghhbbgg_2017_hgg,bbgg_cat12highpurity_2017,/eos/cms/store/group/phys_b2g/shsong/flashggws...,wsig_13TeV:hggpdfsmrel_gghh_2017_bbgg_cat12hig...,None,None
9,shapes,bkg_mass,bbgg_cat12highpurity_2017,/eos/cms/store/group/phys_b2g/shsong/flashggws...,multipdf:CMS_hgg_bbgg_cat12highpurity_2017_13T...,None,None


In [9]:
cat_name = ['cat12highpurity','cat12lowpurity']
# mass_name=['MX1000_MH125','MX500_MH125','MX550_MH125','MX600_MH125','MX650_MH125','MX700_MH125','MX750_MH125','MX800_MH125','MX850_MH125', 'MX1100_MH125', 'MX1200_MH125', 'MX1300_MH125','MX1400_MH125','MX1500_MH125', 'MX1600_MH125', 'MX1700_MH125','MX1800_MH125','MX1900_MH125', 'MX2000_MH125', 'MX2200_MH125', 'MX2600_MH125', 'MX2800_MH125','MX2400_MH125','MX3000_MH125']
# mass_name=['MX1000_MH125']missing for 2018
mass_name=['MX2000_MH125','MX1000_MH125','MX600_MH125','MX3000_MH125']
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for cat in cat_name:
    for mass in mass_name: 
        mass_dir = "datacard_" + mass
        FHSLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_combineFHSL_"+cat+".txt"
        bbggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_bbgg_"+cat+".txt"
        zzggcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/"+mass_dir+"/Datacard_"+mass+"_2018_zzgg_"+cat+".txt"
        FHSLws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/"+mass+"/ws_gghh_"+mass+"_2018_combineFHSL_"+cat
        bbggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/"+mass+"/ws_gghh_"+mass+"_2018_bbgg_"+cat
        zzggws_path="/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/"+mass+"/ws_gghh_"+mass+"_2018_zzgg_"+cat
        FHSLsignalmodel= glob.glob(os.path.join(FHSLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        bbggsignalmodel= glob.glob(os.path.join(bbggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        zzggsignalmodel= glob.glob(os.path.join(zzggws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        print(FHSLws_path)
        print(bbggws_path)
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(destination_text_path):
            os.remove(destination_text_path) 
        print(destination_text_path)
        print(FHSLsignalmodel)
        print(bbggsignalmodel)
        print(zzggsignalmodel)
        if FHSLsignalmodel and bbggsignalmodel and not zzggsignalmodel:
            print("FHSL and bbgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+ "\n")
                lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+ "\n")
                # lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+ "\n")
                lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+ "\n")
                lines[50]=(add_all_line(df_FHSL,46) + df_bbgg.iloc[46][2]+ "\n")
                lines[51]=(add_all_line(df_FHSL,47) + df_bbgg.iloc[47][2]+ "\n")
                
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and bbggsignalmodel and zzggsignalmodel:
            print("FHSL, bbgg, zzgg ",mass,cat)
            print(FHSLcat_path)
            print(bbggcat_path)
            print(zzggcat_path)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_bbgg, 8).replace(" bbgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_bbgg.iloc[15][1].replace("bbgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+"     "+df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_bbgg.iloc[16][1]+"     "+df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_bbgg.iloc[17][1])-1)+"     " +str(int(df_bbgg.iloc[17][1])-2)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_bbgg.iloc[18][1]+"     "+df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_bbgg.iloc[20][2]+"     "+df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_bbgg.iloc[21][2]+"     "+df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_bbgg.iloc[22][2]+"     "+df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_bbgg.iloc[23][2]+"     "+df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_bbgg.iloc[24][2]+"     "+df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_bbgg.iloc[25][2]+"     "+df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_bbgg.iloc[26][2]+"     "+df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_bbgg.iloc[27][2]+"     "+df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_bbgg.iloc[28][2]+"     "+df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_bbgg.iloc[29][2]+"     "+df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_bbgg.iloc[30][2]+"     "+df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_bbgg.iloc[31][2]+"     "+df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_bbgg.iloc[32][2]+"     "+df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_bbgg.iloc[33][2]+"     "+df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_bbgg.iloc[34][2]+"     "+df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_bbgg.iloc[35][2]+"     "+df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_bbgg.iloc[36][2]+"     "+df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_bbgg.iloc[37][2]+"     "+df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_bbgg.iloc[38][2]+"     "+df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_bbgg.iloc[39][2]+"     "+df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_bbgg.iloc[40][2]+"     "+df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_bbgg.iloc[41][2]+"     "+df_zzgg.iloc[41][2]+ "\n")
                lines[46]=(add_all_line(df_FHSL,42) + df_bbgg.iloc[42][2]+"     "+df_zzgg.iloc[42][2]+ "\n")
                # lines[47]=(add_all_line(df_FHSL,43) + df_bbgg.iloc[43][2]+"     "+df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_bbgg.iloc[44][2]+"     "+df_zzgg.iloc[44][2]+ "\n")
                lines[49]=(add_all_line(df_FHSL,45) + df_bbgg.iloc[45][2]+"     "+df_zzgg.iloc[45][2]+ "\n")
                lines[50]=(add_all_line(df_FHSL,46) + df_bbgg.iloc[46][2]+"     "+df_zzgg.iloc[46][2]+ "\n")
                lines[51]=(add_all_line(df_FHSL,47) + df_bbgg.iloc[47][2]+"     "+df_zzgg.iloc[47][2]+ "\n")
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)
        if FHSLsignalmodel and zzggsignalmodel and not bbggsignalmodel:
            print("FHSL and zzgg",mass,cat)
            with open(FHSLcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(destination_text_path, 'w') as destination_file:
                destination_file.write(content.replace("gghh_2018_hgg", "gghhWWgg_2018_hgg"))
            separator = r'(?<!\*)\s+'

            df_FHSL = pd.read_table(FHSLcat_path, sep=separator, header=None, engine='python')
            df_FHSL = df_FHSL.replace('gghh_2018_hgg', 'gghhWWgg_2018_hgg')
            df_bbgg = pd.read_table(bbggcat_path, sep=separator, header=None, engine='python')
            df_bbgg = df_bbgg.replace('gghh_2018_hgg', 'gghhbbgg_2018_hgg')
            df_zzgg = pd.read_table(zzggcat_path, sep=separator, header=None, engine='python')
            df_zzgg = df_zzgg.replace('gghh_2018_hgg', 'gghhzzgg_2018_hgg')
            with open(destination_text_path, 'r') as file:
                lines = file.readlines()
                lines.insert(10, add_all_line(df_zzgg, 8).replace(" zzgg_"+cat+"_2018 "," combineFHSL_"+cat+"_2018 ") + "\n")
                lines[18]=(add_all_line(df_FHSL,15) + df_zzgg.iloc[15][1].replace("zzgg_"+cat+"_2018","combineFHSL_"+cat+"_2018")+ "\n")
                lines[19]=(add_all_line(df_FHSL,16) + df_zzgg.iloc[16][1]+ "\n")
                lines[20]=(add_all_line(df_FHSL,17) + str(int(df_zzgg.iloc[17][1])-1)+ "\n")
                lines[21]=(add_all_line(df_FHSL,18) + df_zzgg.iloc[18][1]+ "\n")
                lines[24]=(add_all_line(df_FHSL,20) + df_zzgg.iloc[20][2]+ "\n")
                lines[25]=(add_all_line(df_FHSL,21) + df_zzgg.iloc[21][2]+ "\n")
                lines[26]=(add_all_line(df_FHSL,22) + df_zzgg.iloc[22][2]+ "\n")
                lines[27]=(add_all_line(df_FHSL,23) + df_zzgg.iloc[23][2]+ "\n")
                lines[28]=(add_all_line(df_FHSL,24) + df_zzgg.iloc[24][2]+ "\n")
                lines[29]=(add_all_line(df_FHSL,25) + df_zzgg.iloc[25][2]+ "\n")
                lines[30]=(add_all_line(df_FHSL,26) + df_zzgg.iloc[26][2]+ "\n")
                lines[31]=(add_all_line(df_FHSL,27) + df_zzgg.iloc[27][2]+ "\n")
                lines[32]=(add_all_line(df_FHSL,28) + df_zzgg.iloc[28][2]+ "\n")
                lines[33]=(add_all_line(df_FHSL,29) + df_zzgg.iloc[29][2]+ "\n")
                lines[34]=(add_all_line(df_FHSL,30) + df_zzgg.iloc[30][2]+ "\n")
                lines[35]=(add_all_line(df_FHSL,31) + df_zzgg.iloc[31][2]+ "\n")
                lines[36]=(add_all_line(df_FHSL,32) + df_zzgg.iloc[32][2]+ "\n")
                lines[37]=(add_all_line(df_FHSL,33) + df_zzgg.iloc[33][2]+ "\n")
                lines[38]=(add_all_line(df_FHSL,34) + df_zzgg.iloc[34][2]+ "\n")
                lines[39]=(add_all_line(df_FHSL,35) + df_zzgg.iloc[35][2]+ "\n")
                lines[40]=(add_all_line(df_FHSL,36) + df_zzgg.iloc[36][2]+ "\n")
                lines[41]=(add_all_line(df_FHSL,37) + df_zzgg.iloc[37][2]+ "\n")
                lines[42]=(add_all_line(df_FHSL,38) + df_zzgg.iloc[38][2]+ "\n")
                lines[43]=(add_all_line(df_FHSL,39) + df_zzgg.iloc[39][2]+ "\n")
                lines[44]=(add_all_line(df_FHSL,40) + df_zzgg.iloc[40][2]+ "\n")
                lines[45]=(add_all_line(df_FHSL,41) + df_zzgg.iloc[41][2]+ "\n")
                lines[46]=(add_all_line(df_FHSL,42) + df_zzgg.iloc[42][2]+ "\n")
                # lines[47]=(add_all_line(df_FHSL,43) + df_zzgg.iloc[43][2]+ "\n")
                lines[48]=(add_all_line(df_FHSL,44) + df_zzgg.iloc[44][2]+ "\n")
                lines[49]=(add_all_line(df_FHSL,45) + df_zzgg.iloc[45][2]+ "\n")
                lines[50]=(add_all_line(df_FHSL,46) + df_zzgg.iloc[46][2]+ "\n")
                lines[51]=(add_all_line(df_FHSL,47) + df_zzgg.iloc[47][2]+ "\n")
                with open(destination_text_path, 'w') as file:
                    file.writelines(lines)

/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/MX2000_MH125/ws_gghh_MX2000_MH125_2018_combineFHSL_cat12highpurity
/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/MX2000_MH125/ws_gghh_MX2000_MH125_2018_bbgg_cat12highpurity
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_MX2000_MH125/Datacard_MX2000_MH125_cat12highpurity.txt
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/MX2000_MH125/ws_gghh_MX2000_MH125_2018_combineFHSL_cat12highpurity/CMS-HGG_sigfit_packaged_combineFHSL_cat12highpurity_2018.root']
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/MX2000_MH125/ws_gghh_MX2000_MH125_2018_bbgg_cat12highpurity/CMS-HGG_sigfit_packaged_bbgg_cat12highpurity_2018.root']
['/eos/cms/store/group/phys_b2g/shsong/flashggws/cat12/2018/MX2000_MH125/ws_gghh_MX2000_MH125_2018_zzgg_cat12highpurity/CMS-HGG_sigfit_packaged_zzgg_cat12highpurity_2018.root']
FHSL, bbgg, zzgg  MX2000_MH125 cat12highpurity
/afs/cern.ch/user/

In [9]:
# run cpcat8txt.sh firstly
cat_name = ["cat8highpurity","cat8lowestpurity","cat8mediumpurity","cat8lowpurity"] 
mass_name=["M250","M260","M270","M280","M300","M320","M350","M400","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900","M1000"]#
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for mass in mass_name:
    cdpath="cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass
    cat1_Txt=os.path.exists("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass+"/Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt") and os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass+"/Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt")>=1024
    cat2_Txt=os.path.exists("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass+"/Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt") and os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass+"/Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt")>=1024
    cat3_Txt=False
    cat4_Txt=os.path.exists("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass+"/Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt") and os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass+"/Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt")>=1024
    cat5_Txt=os.path.exists("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass+"/Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt") and os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass+"/Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt")>=1024
    # if five txt file all exist, then combine 5 txt file
    if cat1_Txt and cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    # if one of the txt file is not exist, then combine other txt file
    elif cat1_Txt and cat2_Txt and cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and cat2_Txt and cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and cat2_Txt and not cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    # if two of the txt file is not exist, then combine other three txt file
    elif cat1_Txt and cat2_Txt and cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and cat2_Txt and not cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and cat2_Txt and cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and cat2_Txt and not cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and not cat2_Txt and cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and cat2_Txt and cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and cat2_Txt and not cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    if not cat1_Txt and not cat2_Txt and not cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and cat2_Txt and not cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and cat2_Txt and not cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and cat2_Txt and cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and not cat2_Txt and not cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and not cat2_Txt and not cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and not cat2_Txt and cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt > Datacard_"+mass+"_cat8.txt"
    elif cat1_Txt and cat2_Txt and not cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat8_"+mass+"=Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt > Datacard_"+mass+"_cat8.txt"
    # if 4 of the txt file is not exist, don't combine txt file 
    elif cat1_Txt and not cat2_Txt and not cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="cp Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and cat2_Txt and not cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="cp Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="cp Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and not cat2_Txt and not cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="cp Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_"+mass+"_cat8.txt"
    elif not cat1_Txt and not cat2_Txt and not cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="cp Datacard_"+mass+"_FH_2016pre_combineFHSL_cat8lowpurity.txt Datacard_"+mass+"_cat8.txt"
    # if 5 of the txt file is not exist, don't combine txt file
    elif not cat1_Txt and not cat2_Txt and not cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine=" "

    GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_cat8_"+mass+" -d Datacard_"+mass+"_cat8.root --run expected    "
    getroot="text2workspace.py Datacard_"+mass+"_cat8.txt -m 125"
    cd.append(cdpath)
    getlimit.append(GetLimit)
    txttoroot.append(getroot)
    Combinecommand.append(Combine)

for i in range(0,len(getlimit)):
    print(cd[i])
    print(Combinecommand[i])
    print(txttoroot[i])
    print(getlimit[i])

cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M250
combineCards.py  Datacard_cat8_M250=Datacard_M250_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_M250_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_M250_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_M250_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_M250_cat8.txt
text2workspace.py Datacard_M250_cat8.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat8_M250 -d Datacard_M250_cat8.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M260
combineCards.py  Datacard_cat8_M260=Datacard_M260_FH_2016pre_combineFHSL_cat8highpurity.txt Datacard_M260_FH_2016pre_combineFHSL_cat8lowestpurity.txt Datacard_M260_FH_2016pre_combineFHSL_cat8mediumpurity.txt Datacard_M260_FH_2016pre_combineFHSL_cat8lowpurity.txt > Datacard_M260_cat8.txt
text2workspace.py Datacard_M260_cat8.txt -m 125
combine -M Asymp

In [5]:
# run cpcat2txt.sh firstly
cat_name = ["cat2highpt","cat2lowpt","cat2highpurity","cat2mediumpurity","cat2lowpurity"] 
mass_name=["M250","M260","M270","M280","M300","M320","M350","M400","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900","M1000"]#
#combine command
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for mass in mass_name:
    cdpath="cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_"+mass
    cat1_Txt=os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_"+mass+"/Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt")>=1024
    cat2_Txt=os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_"+mass+"/Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt")>=1024
    cat3_Txt=os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_"+mass+"/Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt")>=1024
    cat4_Txt=os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_"+mass+"/Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt")>=1024
    cat5_Txt=os.path.getsize("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_"+mass+"/Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt")>=1024
    # if five txt file all exist, then combine 5 txt file
    if cat1_Txt and cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    # if one of the txt file is not exist, then combine other txt file
    elif cat1_Txt and cat2_Txt and cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and cat2_Txt and cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and cat2_Txt and not cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    # if two of the txt file is not exist, then combine other three txt file
    elif cat1_Txt and cat2_Txt and cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and cat2_Txt and not cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and cat2_Txt and cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and cat2_Txt and not cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and not cat2_Txt and cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and cat2_Txt and cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and cat2_Txt and not cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    if not cat1_Txt and not cat2_Txt and not cat3_Txt and cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and cat2_Txt and not cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and cat2_Txt and not cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and cat2_Txt and cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and not cat2_Txt and not cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and not cat2_Txt and not cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and not cat2_Txt and cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt > Datacard_"+mass+"_cat2.txt"
    elif cat1_Txt and cat2_Txt and not cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="combineCards.py  Datacard_cat2_"+mass+"=Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt > Datacard_"+mass+"_cat2.txt"
    # if 4 of the txt file is not exist, don't combine txt file 
    elif cat1_Txt and not cat2_Txt and not cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="cp Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpt.txt Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and cat2_Txt and not cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="cp Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpt.txt Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and not cat2_Txt and cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine="cp Datacard_"+mass+"_SL_2018_combineFHSL_cat2highpurity.txt Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and not cat2_Txt and not cat3_Txt and cat4_Txt and not cat5_Txt:
        Combine="cp Datacard_"+mass+"_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_"+mass+"_cat2.txt"
    elif not cat1_Txt and not cat2_Txt and not cat3_Txt and not cat4_Txt and cat5_Txt:
        Combine="cp Datacard_"+mass+"_SL_2018_combineFHSL_cat2lowpurity.txt Datacard_"+mass+"_cat2.txt"
    # if 5 of the txt file is not exist, don't combine txt file
    elif not cat1_Txt and not cat2_Txt and not cat3_Txt and not cat4_Txt and not cat5_Txt:
        Combine=" "

    GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_cat2_"+mass+" -d Datacard_"+mass+"_cat2.root --run expected    "
    getroot="text2workspace.py Datacard_"+mass+"_cat2.txt -m 125"
    cd.append(cdpath)
    getlimit.append(GetLimit)
    txttoroot.append(getroot)
    Combinecommand.append(Combine)

for i in range(0,len(getlimit)):
    print(cd[i])
    print(Combinecommand[i])
    print(txttoroot[i])
    print(getlimit[i])

cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_M250
combineCards.py  Datacard_cat2_M250=Datacard_M250_SL_2018_combineFHSL_cat2highpt.txt Datacard_M250_SL_2018_combineFHSL_cat2lowpt.txt Datacard_M250_SL_2018_combineFHSL_cat2highpurity.txt Datacard_M250_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_M250_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_M250_cat2.txt
text2workspace.py Datacard_M250_cat2.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat2_M250 -d Datacard_M250_cat2.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_M260
combineCards.py  Datacard_cat2_M260=Datacard_M260_SL_2018_combineFHSL_cat2highpt.txt Datacard_M260_SL_2018_combineFHSL_cat2lowpt.txt Datacard_M260_SL_2018_combineFHSL_cat2highpurity.txt Datacard_M260_SL_2018_combineFHSL_cat2mediumpurity.txt Datacard_M260_SL_2018_combineFHSL_cat2lowpurity.txt > Datacard_M260_cat2.txt
text2works

In [12]:
lines[43]

'CMS_MET_2018                                        lnN           -               -              \n'

In [16]:
cat_name = ["cat3highpt","cat3lowpt"] #lowmass
# mass_name=["M250","M260","M270","M280","M300","M320","M350","M400","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900","M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200"]
mass_name=["M2400","M2600","M2800","M3000"]#
# mass_name=["M800","M850","M900"]#
# mass_name = []

for cat in cat_name:
    cat3_mass = []
    for mass in mass_name:    
        FHcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/Datacard_"+mass+"_FH_2016pre_combineFHSL_"+cat+".txt"
        SLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/Datacard_"+mass+"_SL_2016pre_combineFHSL_"+cat+".txt"
        FHws_path="/eos/user/s/shsong/HHWWgg_limit/optimise/cat3/2016pre/flashgginput/"+mass+"_FH/ws_gghhFH_"+mass+"_FH_2016pre_combineFHSL_"+cat
        SLws_path="/eos/user/s/shsong/HHWWgg_limit/optimise/cat3/2016pre/flashgginput/"+mass+"_SL/ws_gghhSL_"+mass+"_SL_2016pre_combineFHSL_"+cat
        SLsignalmodel= glob.glob(os.path.join(SLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        FHsignalmodel= glob.glob(os.path.join(FHws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        print(FHws_path)
        print(SLws_path)
        mass_dir = "datacard_" + mass
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        FHSL_destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(FHSL_destination_text_path):
            os.remove(FHSL_destination_text_path) 
        print(FHSL_destination_text_path)
        print(FHsignalmodel)
        print(SLsignalmodel)
        if FHsignalmodel and SLsignalmodel:
            with open(FHcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(FHSL_destination_text_path, 'w') as destination_file:
                destination_file.write(content)
            separator = r'(?<!\*)\s+'


            # 读取文本文件并使用正则表达式作为分隔符
            print(FHcat_path)
            print(SLcat_path)
            df_FH = pd.read_table(FHcat_path, sep=separator, header=None, engine='python')
            df_SL = pd.read_table(SLcat_path, sep=separator, header=None, engine='python')

            with open(FHSL_destination_text_path, 'r') as file:
                lines = file.readlines()
            lines[17]=(add_all_line(df_FH,15) + df_SL.iloc[15][2]+ "\n")
            lines[18]=(add_all_line(df_FH,16) + df_SL.iloc[16][1]+ "\n")
            lines[19]=(add_all_line(df_FH,17) + str(int(df_SL.iloc[17][1])-1)+ "\n")
            lines[20]=(add_all_line(df_FH,18) + df_SL.iloc[18][1] + "\n")
            lines.insert(10, add_all_line(df_SL, 8) + "\n")

            lines.insert((len(lines)-3),add_all_line(df_SL,df_SL.index[-4]) + "\n")
            lines.insert((len(lines)-3),add_all_line(df_SL,df_SL.index[-3]) + "\n")
            lines.insert((len(lines)-1),add_all_line(df_SL,df_SL.index[-1]) + "\n")

            lines[24]=(add_all_line(df_FH,20) + df_SL.iloc[20][2] + "\n")
            lines[25]=(add_all_line(df_FH,21) + df_SL.iloc[21][2] + "\n")
            lines[26]=(add_all_line(df_FH,22) + df_SL.iloc[22][2] + "\n")
            lines[27]=(add_all_line(df_FH,23) + df_SL.iloc[23][2] + "\n")
            lines[28]=(add_all_line(df_FH,24) + df_SL.iloc[24][2] + "\n")
            lines[29]=(add_all_line(df_FH,25) + df_SL.iloc[25][2] + "\n")
            lines[30]=(add_all_line(df_FH,26) + df_SL.iloc[26][2] + "\n")
            lines[31]=(add_all_line(df_FH,27) + df_SL.iloc[27][2] + "\n")
            lines[32]=(add_all_line(df_FH,28) + df_SL.iloc[28][2] + "\n")
            lines[33]=(add_all_line(df_FH,29) + df_SL.iloc[29][2] + "\n")
            lines[34]=(add_all_line(df_FH,30) + df_SL.iloc[30][2] + "\n")
            lines[35]=(add_all_line(df_FH,31) + df_SL.iloc[31][2] + "\n")
            lines[36]=(add_all_line(df_FH,32) + df_SL.iloc[32][2] + "\n")
            lines[37]=(add_all_line(df_FH,33) + df_SL.iloc[33][2] + "\n")
            lines[38]=(add_all_line(df_FH,34) + df_SL.iloc[34][2] + "\n")
            lines[39]=(add_all_line(df_FH,35) + df_SL.iloc[35][2] + "\n")
            lines[40]=(add_all_line(df_FH,36) + df_SL.iloc[36][2] + "\n")
            lines[41]=(add_all_line(df_FH,37) + df_SL.iloc[37][2] + "\n")
            # lines[42]=(add_all_line(df_FH,38) + df_SL.iloc[38][2] + "\n")
            lines[43]=(add_all_line(df_FH,39) + df_SL.iloc[39][2] + "\n")
            lines[44]=(add_all_line(df_FH,40) + df_SL.iloc[40][2] + "\n")
            lines[45]=(add_all_line(df_FH,41) + df_SL.iloc[41][2] + "\n")    
            lines[46]=(add_all_line(df_FH,42) + df_SL.iloc[42][2] + "\n")    
            # lines[47]=(add_all_line(df_FH,43) + df_SL.iloc[43][2] + "\n")
            # lines[48]=(add_all_line(df_FH,44) + df_SL.iloc[44][2] + "\n")
            # lines[49]=(add_all_line(df_FH,45) + df_SL.iloc[45][2] + "\n")
            # lines[52]=(add_all_line(df_FH,48) + df_SL.iloc[48][2] + "\n")
            # lines[53]=(add_all_line(df_FH,49) + df_SL.iloc[49][2] + "\n")
            # lines[54]=(add_all_line(df_FH,50) + df_SL.iloc[50][2] + "\n")
            # lines[55]=(add_all_line(df_FH,51) + df_SL.iloc[51][2] + "\n")
            with open(FHSL_destination_text_path, 'w') as file:
                file.writelines(lines)
        if FHsignalmodel and not SLsignalmodel:
            subprocess.run(["cp", FHcat_path, FHSL_destination_text_path])
        if not FHsignalmodel and SLsignalmodel:
            subprocess.run(["cp", SLcat_path, FHSL_destination_text_path])
        if not FHsignalmodel and not SLsignalmodel:
            print("No signal model")

/eos/user/s/shsong/HHWWgg_limit/optimise/cat3/2016pre/flashgginput/M2400_FH/ws_gghhFH_M2400_FH_2016pre_combineFHSL_cat3highpt
/eos/user/s/shsong/HHWWgg_limit/optimise/cat3/2016pre/flashgginput/M2400_SL/ws_gghhSL_M2400_SL_2016pre_combineFHSL_cat3highpt
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M2400/Datacard_M2400_cat3highpt.txt
['/eos/user/s/shsong/HHWWgg_limit/optimise/cat3/2016pre/flashgginput/M2400_FH/ws_gghhFH_M2400_FH_2016pre_combineFHSL_cat3highpt/CMS-HGG_sigfit_packaged_combineFHSL_cat3highpt_2016pre.root']
['/eos/user/s/shsong/HHWWgg_limit/optimise/cat3/2016pre/flashgginput/M2400_SL/ws_gghhSL_M2400_SL_2016pre_combineFHSL_cat3highpt/CMS-HGG_sigfit_packaged_combineFHSL_cat3highpt_2016pre.root']
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/Datacard_M2400_FH_2016pre_combineFHSL_cat3highpt.txt
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/Datacard_M2400_SL_2016pre_combineFHSL_cat3hi

In [50]:
#get commands
mass_name=["M250","M260","M270","M280","M300","M320","M350","M400","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900","M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]#

# mass_name = []
cd=[]
getlimit=[]
Combinecommand=[]
txttoroot=[]
for mass in mass_name:
    datacardpath="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass
    cat0_txt= glob.glob(os.path.join(datacardpath, 'Datacard_'+mass+'_'+cat_name[0]+'.txt'))
    cat1_txt= glob.glob(os.path.join(datacardpath, 'Datacard_'+mass+'_'+cat_name[1]+'.txt'))
    # os.chdir("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass)
    cdpath="cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass
    if len(cat0_txt) and len(cat1_txt):
        Combine="combineCards.py  "+cat_name[0].split("high")[0]+"_"+mass+"=Datacard_"+mass+"_"+cat_name[0]+".txt Datacard_"+mass+"_"+cat_name[1]+".txt > Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        # combinecard.append(Combine)
        # getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        # subprocess.call(Combine, shell=True, stdout=subprocess.PIPE)
        Combinecommand.append(Combine)
    if len(cat0_txt) and not len(cat1_txt):
        Combine= "cp "+cat0_txt[0]+" Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        cd.append(cdpath)
        Combinecommand.append(Combine)
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        txttoroot.append(getroot)

        # subprocess.run(["cp", cat0_txt, "Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"])
    if not len(cat0_txt) and len(cat1_txt):
        Combine= "cp "+cat1_txt[0]+" Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        cd.append(cdpath)
        Combinecommand.append(Combine)
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        txttoroot.append(getroot)
        # subprocess.run(["cp", cat1_txt, "Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"])
    if not len(cat0_txt) and not len(cat1_txt):
        print("#no datacard")
for i in range(0,len(getlimit)):
    print(cd[i])
    print(Combinecommand[i])
    print(txttoroot[i])
    print(getlimit[i])

    

cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_M250
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_M250/Datacard_M250_cat3lowpt.txt Datacard_M250_cat3.txt
text2workspace.py Datacard_M250_cat3.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat3_M250 -d Datacard_M250_cat3.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_M260
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_M260/Datacard_M260_cat3lowpt.txt Datacard_M260_cat3.txt
text2workspace.py Datacard_M260_cat3.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat3_M260 -d Datacard_M260_cat3.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/datacard_M270
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2018datacard/data

In [5]:
# cat_name = ["cat5lowpt"] #"M250"不行
cat_name = ["cat5highpt","cat5lowpt"] #"
# mass_name=["M250","M270","M280","M300","M320","M350","M400","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900"]
mass_name=["M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]#

# mass_name=["M270"]#
for cat in cat_name:
    cat5_mass = []
    for mass in mass_name:    
        FHcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/Datacard_"+mass+"_FH_2016post_combineFHSL_"+cat+".txt"
        SLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/Datacard_"+mass+"_SL_2016post_combineFHSL_"+cat+".txt"
        FHws_path="/eos/user/s/shsong/HHWWgg_limit/optimise/cat5/2016post/flashgginput/"+mass+"_FH/ws_gghhFH_"+mass+"_FH_2016post_combineFHSL_"+cat
        SLws_path="/eos/user/s/shsong/HHWWgg_limit/optimise/cat5/2016post/flashgginput/"+mass+"_SL/ws_gghhSL_"+mass+"_SL_2016post_combineFHSL_"+cat
        SLsignalmodel= glob.glob(os.path.join(SLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        FHsignalmodel= glob.glob(os.path.join(FHws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        print(FHws_path)
        print(SLws_path)
        mass_dir = "datacard_" + mass
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        FHSL_destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(FHSL_destination_text_path):
            os.remove(FHSL_destination_text_path) 
        print(FHSL_destination_text_path)
        print(FHsignalmodel)
        print(SLsignalmodel)
        if FHsignalmodel and SLsignalmodel:
            with open(FHcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(FHSL_destination_text_path, 'w') as destination_file:
                destination_file.write(content)
            separator = r'(?<!\*)\s+'


            # 读取文本文件并使用正则表达式作为分隔符
            df_FH = pd.read_table(FHcat_path, sep=separator, header=None, engine='python')
            df_SL = pd.read_table(SLcat_path, sep=separator, header=None, engine='python')

            with open(FHSL_destination_text_path, 'r') as file:
                lines = file.readlines()
            lines[17]=(add_all_line(df_FH,15) + df_SL.iloc[15][2]+ "\n")
            lines[18]=(add_all_line(df_FH,16) + df_SL.iloc[16][1]+ "\n")
            lines[19]=(add_all_line(df_FH,17) + str(int(df_SL.iloc[17][1])-1)+ "\n")
            lines[20]=(add_all_line(df_FH,18) + df_SL.iloc[18][1] + "\n")
            lines.insert(10, add_all_line(df_SL, 8) + "\n")

            lines.insert((len(lines)-3),add_all_line(df_SL,df_SL.index[-4]) + "\n")
            lines.insert((len(lines)-3),add_all_line(df_SL,df_SL.index[-3]) + "\n")
            lines.insert((len(lines)-1),add_all_line(df_SL,df_SL.index[-1]) + "\n")

            lines[24]=(add_all_line(df_FH,20) + df_SL.iloc[20][2] + "\n")
            lines[25]=(add_all_line(df_FH,21) + df_SL.iloc[21][2] + "\n")
            lines[26]=(add_all_line(df_FH,22) + df_SL.iloc[22][2] + "\n")
            lines[27]=(add_all_line(df_FH,23) + df_SL.iloc[23][2] + "\n")
            lines[28]=(add_all_line(df_FH,24) + df_SL.iloc[24][2] + "\n")
            lines[29]=(add_all_line(df_FH,25) + df_SL.iloc[25][2] + "\n")
            lines[30]=(add_all_line(df_FH,26) + df_SL.iloc[26][2] + "\n")
            lines[31]=(add_all_line(df_FH,27) + df_SL.iloc[27][2] + "\n")
            lines[32]=(add_all_line(df_FH,28) + df_SL.iloc[28][2] + "\n")
            lines[33]=(add_all_line(df_FH,29) + df_SL.iloc[29][2] + "\n")
            lines[34]=(add_all_line(df_FH,30) + df_SL.iloc[30][2] + "\n")
            lines[35]=(add_all_line(df_FH,31) + df_SL.iloc[31][2] + "\n")
            lines[36]=(add_all_line(df_FH,32) + df_SL.iloc[32][2] + "\n")
            lines[37]=(add_all_line(df_FH,33) + df_SL.iloc[33][2] + "\n")
            lines[38]=(add_all_line(df_FH,34) + df_SL.iloc[34][2] + "\n")
            lines[39]=(add_all_line(df_FH,35) + df_SL.iloc[35][2] + "\n")
            lines[40]=(add_all_line(df_FH,36) + df_SL.iloc[36][2] + "\n")
            lines[41]=(add_all_line(df_FH,37) + df_SL.iloc[37][2] + "\n")
            lines[42]=(add_all_line(df_FH,38) + df_SL.iloc[38][2] + "\n")
            # lines[43]=(add_all_line(df_FH,39) + df_SL.iloc[39][2] + "\n")
            lines[44]=(add_all_line(df_FH,40) + df_SL.iloc[40][2] + "\n")
            lines[45]=(add_all_line(df_FH,41) + df_SL.iloc[41][2] + "\n")    
            lines[46]=(add_all_line(df_FH,42) + df_SL.iloc[42][2] + "\n")    
            lines[47]=(add_all_line(df_FH,43) + df_SL.iloc[43][2] + "\n")
            # lines[48]=(add_all_line(df_FH,44) + df_SL.iloc[44][2] + "\n")
            # lines[49]=(add_all_line(df_FH,45) + df_SL.iloc[45][2] + "\n")
            # lines[50]=(add_all_line(df_FH,46) + df_SL.iloc[46][2] + "\n")
            with open(FHSL_destination_text_path, 'w') as file:
                file.writelines(lines)
        if FHsignalmodel and not SLsignalmodel:
            subprocess.run(["cp", FHcat_path, FHSL_destination_text_path])
        if not FHsignalmodel and SLsignalmodel:
            subprocess.run(["cp", SLcat_path, FHSL_destination_text_path])
        if not FHsignalmodel and not SLsignalmodel:
            print("No signal model")

/eos/user/s/shsong/HHWWgg_limit/optimise/cat5/2016post/flashgginput/M1000_FH/ws_gghhFH_M1000_FH_2016post_combineFHSL_cat5highpt
/eos/user/s/shsong/HHWWgg_limit/optimise/cat5/2016post/flashgginput/M1000_SL/ws_gghhSL_M1000_SL_2016post_combineFHSL_cat5highpt
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_M1000/Datacard_M1000_cat5highpt.txt
['/eos/user/s/shsong/HHWWgg_limit/optimise/cat5/2016post/flashgginput/M1000_FH/ws_gghhFH_M1000_FH_2016post_combineFHSL_cat5highpt/CMS-HGG_sigfit_packaged_combineFHSL_cat5highpt_2016post.root']
['/eos/user/s/shsong/HHWWgg_limit/optimise/cat5/2016post/flashgginput/M1000_SL/ws_gghhSL_M1000_SL_2016post_combineFHSL_cat5highpt/CMS-HGG_sigfit_packaged_combineFHSL_cat5highpt_2016post.root']
/eos/user/s/shsong/HHWWgg_limit/optimise/cat5/2016post/flashgginput/M1100_FH/ws_gghhFH_M1100_FH_2016post_combineFHSL_cat5highpt
/eos/user/s/shsong/HHWWgg_limit/optimise/cat5/2016post/flashgginput/M1100_SL/ws_gghhSL_M1100_SL_20

In [6]:
#get commands
# mass_name=["M250","M270","M280","M300","M320","M350","M400","M450","M500","M600","M650","M700","M750","M800","M850","M900","M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]
mass_name=["M250","M270","M280","M300","M320","M350","M400","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900","M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]
cat_name = ["cat5highpt","cat5lowpt"] 
cd=[]
getlimit=[]
Combinecommand=[]
txttoroot=[]
for mass in mass_name:
    datacardpath="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass
    cat0_txt= glob.glob(os.path.join(datacardpath, 'Datacard_'+mass+'_'+cat_name[0]+'.txt'))
    cat1_txt= glob.glob(os.path.join(datacardpath, 'Datacard_'+mass+'_'+cat_name[1]+'.txt'))
    # os.chdir("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass)
    cdpath="cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass
    if len(cat0_txt) and len(cat1_txt):
        Combine="combineCards.py  "+cat_name[0].split("high")[0]+"_"+mass+"=Datacard_"+mass+"_"+cat_name[0]+".txt Datacard_"+mass+"_"+cat_name[1]+".txt > Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        # combinecard.append(Combine)
        # getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        # subprocess.call(Combine, shell=True, stdout=subprocess.PIPE)
        Combinecommand.append(Combine)
    if len(cat0_txt) and not len(cat1_txt):
        Combine= "cp "+cat0_txt[0]+" Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        cd.append(cdpath)
        Combinecommand.append(Combine)
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        txttoroot.append(getroot)

        # subprocess.run(["cp", cat0_txt, "Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"])
    if not len(cat0_txt) and len(cat1_txt):
        Combine= "cp "+cat1_txt[0]+" Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        cd.append(cdpath)
        Combinecommand.append(Combine)
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        txttoroot.append(getroot)
        # subprocess.run(["cp", cat1_txt, "Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"])
    if not len(cat0_txt) and not len(cat1_txt):
        print("#no datacard")
for i in range(0,len(getlimit)):
    print(cd[i])
    print(Combinecommand[i])
    print(txttoroot[i])
    print(getlimit[i])

    

#no datacard
#no datacard
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M280
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M280/Datacard_M280_cat5lowpt.txt Datacard_M280_cat5.txt
text2workspace.py Datacard_M280_cat5.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat5_M280 -d Datacard_M280_cat5.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M300
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M300/Datacard_M300_cat5lowpt.txt Datacard_M300_cat5.txt
text2workspace.py Datacard_M300_cat5.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat5_M300 -d Datacard_M300_cat5.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M320
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/f

In [4]:
# cat_name = ["cat7"] 
cat_name = ["cat7"] 
mass_name = ["M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]
mass_name=["M250","M260","M270","M280","M300","M320","M350","M400","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900","M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]
for cat in cat_name:
    cat7_mass = []
    for mass in mass_name:    
        FHcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/Datacard_"+mass+"_FH_2016pre_"+cat+".txt"
        SLcat_path="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/Datacard_"+mass+"_SL_2016pre_"+cat+".txt"
        FHws_path="/eos/user/s/shsong/HHWWgg_limit/optimise/cat7/2016pre/flashgginput/"+mass+"_FH/ws_gghhFH_"+mass+"_FH_2016pre_"+cat
        SLws_path="/eos/user/s/shsong/HHWWgg_limit/optimise/cat7/2016pre/flashgginput/"+mass+"_SL/ws_gghhSL_"+mass+"_SL_2016pre_"+cat
        SLsignalmodel= glob.glob(os.path.join(SLws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        FHsignalmodel= glob.glob(os.path.join(FHws_path, 'CMS-HGG_sigfit_packaged_*.root'))
        mass_dir = "datacard_" + mass
        if not os.path.exists(mass_dir):
            os.makedirs(mass_dir)
        FHSL_destination_text_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/" + mass_dir + "/Datacard_" + mass + "_" + cat + ".txt"
        if os.path.exists(FHSL_destination_text_path):
            os.remove(FHSL_destination_text_path) 
        print(FHSL_destination_text_path)
        print(FHsignalmodel)
        print(SLsignalmodel)
        if FHsignalmodel and SLsignalmodel:
            with open(FHcat_path, 'r') as source_file:
                        content = source_file.read()
            with open(FHSL_destination_text_path, 'w') as destination_file:
                destination_file.write(content)
            separator = r'(?<!\*)\s+'


            # 读取文本文件并使用正则表达式作为分隔符
            df_FH = pd.read_table(FHcat_path, sep=separator, header=None, engine='python')
            df_SL = pd.read_table(SLcat_path, sep=separator, header=None, engine='python')

            with open(FHSL_destination_text_path, 'r') as file:
                lines = file.readlines()
            lines[17]=(add_all_line(df_FH,15) + df_SL.iloc[15][2]+ "\n")
            lines[18]=(add_all_line(df_FH,16) + df_SL.iloc[16][1]+ "\n")
            lines[19]=(add_all_line(df_FH,17) + str(int(df_SL.iloc[17][1])-1)+ "\n")
            lines[20]=(add_all_line(df_FH,18) + df_SL.iloc[18][1] + "\n")
            lines.insert(10, add_all_line(df_SL, 8) + "\n")

            lines.insert((len(lines)-3),add_all_line(df_SL,df_SL.index[-4]) + "\n")
            lines.insert((len(lines)-3),add_all_line(df_SL,df_SL.index[-3]) + "\n")
            lines.insert((len(lines)-1),add_all_line(df_SL,df_SL.index[-1]) + "\n")

            lines[24]=(add_all_line(df_FH,20) + df_SL.iloc[20][2] + "\n")
            lines[25]=(add_all_line(df_FH,21) + df_SL.iloc[21][2] + "\n")
            lines[26]=(add_all_line(df_FH,22) + df_SL.iloc[22][2] + "\n")
            lines[27]=(add_all_line(df_FH,23) + df_SL.iloc[23][2] + "\n")
            lines[28]=(add_all_line(df_FH,24) + df_SL.iloc[24][2] + "\n")
            lines[29]=(add_all_line(df_FH,25) + df_SL.iloc[25][2] + "\n")
            lines[30]=(add_all_line(df_FH,26) + df_SL.iloc[26][2] + "\n")
            lines[31]=(add_all_line(df_FH,27) + df_SL.iloc[27][2] + "\n")
            lines[32]=(add_all_line(df_FH,28) + df_SL.iloc[28][2] + "\n")
            lines[33]=(add_all_line(df_FH,29) + df_SL.iloc[29][2] + "\n")
            lines[34]=(add_all_line(df_FH,30) + df_SL.iloc[30][2] + "\n")
            lines[35]=(add_all_line(df_FH,31) + df_SL.iloc[31][2] + "\n")
            lines[36]=(add_all_line(df_FH,32) + df_SL.iloc[32][2] + "\n")
            lines[37]=(add_all_line(df_FH,33) + df_SL.iloc[33][2] + "\n")
            lines[38]=(add_all_line(df_FH,34) + df_SL.iloc[34][2] + "\n")
            lines[39]=(add_all_line(df_FH,35) + df_SL.iloc[35][2] + "\n")
            lines[40]=(add_all_line(df_FH,36) + df_SL.iloc[36][2] + "\n")
            lines[41]=(add_all_line(df_FH,37)+ "\n")
            lines[42]=(add_all_line(df_FH,38) + df_SL.iloc[38][2] + "\n")
            lines[43]=(add_all_line(df_FH,39) + df_SL.iloc[39][2] + "\n")
            lines[44]=(add_all_line(df_FH,40) + df_SL.iloc[40][2] + "\n")
            lines[45]=(add_all_line(df_FH,41) + df_SL.iloc[41][2] + "\n")    
            with open(FHSL_destination_text_path, 'w') as file:
                file.writelines(lines)
        if FHsignalmodel and not SLsignalmodel:
            cat7_mass.append(mass)
            subprocess.run(["cp", FHcat_path, FHSL_destination_text_path])
        if not FHsignalmodel and SLsignalmodel:
            cat7_mass.append(mass)
            subprocess.run(["cp", SLcat_path, FHSL_destination_text_path])
        if not FHsignalmodel and not SLsignalmodel:
            cat7_mass.append(-999)
            print("no signal model")


/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M250/Datacard_M250_cat7.txt
[]
[]
no signal model
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M260/Datacard_M260_cat7.txt
['/eos/user/s/shsong/HHWWgg_limit/optimise/cat7/2016pre/flashgginput/M260_FH/ws_gghhFH_M260_FH_2016pre_cat7/CMS-HGG_sigfit_packaged_cat7_2016pre.root']
[]
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M270/Datacard_M270_cat7.txt
[]
[]
no signal model
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M280/Datacard_M280_cat7.txt
['/eos/user/s/shsong/HHWWgg_limit/optimise/cat7/2016pre/flashgginput/M280_FH/ws_gghhFH_M280_FH_2016pre_cat7/CMS-HGG_sigfit_packaged_cat7_2016pre.root']
[]
/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M300/Datacard_M300_cat7.txt
['/eos/user/s/shsong/HHWWgg_lim

In [5]:
#get commands
cd=[]
getlimit1=[]
getlimit2=[]
Combinecommand=[]
txttoroot2=[]
txttoroot1=[]
for mass in mass_name:
    datacardpath="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass
    cat0_txt= glob.glob(os.path.join(datacardpath, 'Datacard_'+mass+'_cat1.txt'))
    cat1_txt= glob.glob(os.path.join(datacardpath, 'Datacard_'+mass+'_cat7.txt'))
    # os.chdir("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass)
    cdpath="cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_"+mass
    if len(cat0_txt) and len(cat1_txt):
        GetLimit1="combine -M AsymptoticLimits -m 125 -n combineFHSL_cat1_"+mass+" -d Datacard_"+mass+"_cat1.root --run expected    "
        GetLimit2="combine -M AsymptoticLimits -m 125 -n combineFHSL_cat7_"+mass+" -d Datacard_"+mass+"_cat7.root --run expected    "

        getroot1="text2workspace.py Datacard_"+mass+"_cat1.txt -m 125"
        getroot2="text2workspace.py Datacard_"+mass+"_cat7.txt -m 125"
        cd.append(cdpath)
        getlimit2.append(GetLimit2)
        getlimit1.append(GetLimit1)
        txttoroot1.append(getroot1)
        txttoroot2.append(getroot2)
        # subprocess.call(Combine, shell=True, stdout=subprocess.PIPE)
    if len(cat0_txt) and not len(cat1_txt):
        GetLimit1="combine -M AsymptoticLimits -m 125 -n combineFHSL_cat1_"+mass+" -d Datacard_"+mass+"_cat1.root --run expected    "
        GetLimit2="  "

        getroot1="text2workspace.py Datacard_"+mass+"_cat1.txt -m 125"
        getroot2="   "
        cd.append(cdpath)
        getlimit2.append(GetLimit2)
        getlimit1.append(GetLimit1)
        txttoroot1.append(getroot1)
        txttoroot2.append(getroot2)
    if not len(cat0_txt) and len(cat1_txt):
        GetLimit1="   "
        GetLimit2="combine -M AsymptoticLimits -m 125 -n combineFHSL_cat7_"+mass+" -d Datacard_"+mass+"_cat7.root --run expected    "

        getroot1="  "
        getroot2="text2workspace.py Datacard_"+mass+"_cat7.txt -m 125"
        cd.append(cdpath)
        getlimit2.append(GetLimit2)
        getlimit1.append(GetLimit1)
        txttoroot1.append(getroot1)
        txttoroot2.append(getroot2)
    if not len(cat0_txt) and not len(cat1_txt):
        print("#no datacard")
for i in range(0,len(getlimit1)):
    print(cd[i])
    # print(txttoroot1[i])
    print(txttoroot2[i])
    # print(getlimit1[i])
    print(getlimit2[i])



cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M250
   
  
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M260
text2workspace.py Datacard_M260_cat7.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat7_M260 -d Datacard_M260_cat7.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M270
   
  
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M280
text2workspace.py Datacard_M280_cat7.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat7_M280 -d Datacard_M280_cat7.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016predatacard/datacard_M300
text2workspace.py Datacard_M300_cat7.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat7_M300 -d Datacard_M300_cat7.root --run expected    
cd /afs/c

In [8]:
#combine command
mass_name = ["M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
for mass in mass_name:
    cdpath="cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_"+mass
    cat1_Txt= len(glob.glob(os.path.join("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_"+mass, 'Datacard_'+mass+'_cat1.txt')))
    cat3_Txt= len(glob.glob(os.path.join("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_"+mass, 'Datacard_'+mass+'_cat3.txt')))
    cat5_Txt= len(glob.glob(os.path.join("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_"+mass, 'Datacard_'+mass+'_cat5.txt')))
    cat7_Txt= len(glob.glob(os.path.join("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_"+mass, 'Datacard_'+mass+'_cat7.txt')))
    if cat1_Txt and cat3_Txt and cat5_Txt and cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat1.txt Datacard_"+mass+"_cat3.txt Datacard_"+mass+"_cat5.txt Datacard_"+mass+"_cat7.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d Datacard_"+mass+"_2017.root --run expected    "
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        Combinecommand.append(Combine)
    if cat1_Txt and cat3_Txt and cat5_Txt and not cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat1.txt Datacard_"+mass+"_cat3.txt Datacard_"+mass+"_cat5.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d Datacard_"+mass+"_2017.root --run expected    "
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        Combinecommand.append(Combine)
    if cat1_Txt and cat3_Txt and not cat5_Txt and cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat1.txt Datacard_"+mass+"_cat3.txt Datacard_"+mass+"_cat7.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d Datacard_"+mass+"_2017.root --run expected    "
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        Combinecommand.append(Combine)
    if cat1_Txt and not cat3_Txt and cat5_Txt and cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat1.txt Datacard_"+mass+"_cat5.txt Datacard_"+mass+"_cat7.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d Datacard_"+mass+"_2017.root --run expected    "
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        Combinecommand.append(Combine)
    if not cat1_Txt and cat3_Txt and cat5_Txt and cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat3.txt Datacard_"+mass+"_cat5.txt Datacard_"+mass+"_cat7.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d Datacard_"+mass+"_2017.root --run expected    "
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        Combinecommand.append(Combine)
        
    if cat1_Txt and cat3_Txt and not cat5_Txt and not cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat1.txt Datacard_"+mass+"_cat3.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d Datacard_"+mass+"_2017.root --run expected    "
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        Combinecommand.append(Combine)
    if cat1_Txt and not cat3_Txt and cat5_Txt and not cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat1.txt Datacard_"+mass+"_cat5.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d Datacard_"+mass+"_2017.root --run expected    "
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        Combinecommand.append(Combine)
    if not cat1_Txt and cat3_Txt and cat5_Txt and not cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat3.txt Datacard_"+mass+"_cat5.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d  Datacard_"+mass+"_2017.root --run expected"
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        Combinecommand.append(Combine)
        txttoroot.append(getroot)
    if cat1_Txt and not cat3_Txt and not cat5_Txt and cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat1.txt Datacard_"+mass+"_cat7.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d Datacard_"+mass+"_2017.root --run expected"
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        Combinecommand.append(Combine)
    if not cat1_Txt and cat3_Txt and not cat5_Txt and cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat3.txt Datacard_"+mass+"_cat7.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d  Datacard_"+mass+"_2017.root --run expected"
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        Combinecommand.append(Combine)
        txttoroot.append(getroot)
    if not cat1_Txt and not cat3_Txt and cat5_Txt and cat7_Txt:
        Combine="combineCards.py  Datacard_"+mass+"=Datacard_"+mass+"_cat5.txt Datacard_"+mass+"_cat7.txt > Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d  Datacard_"+mass+"_2017.root --run expected"
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        Combinecommand.append(Combine)
        txttoroot.append(getroot)
    if not cat1_Txt and not cat3_Txt and not cat5_Txt and cat7_Txt:
        Combine="cp Datacard_"+mass+"_cat7.txt Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d  Datacard_"+mass+"_2017.root --run expected"
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        Combinecommand.append(Combine)
        txttoroot.append(getroot)
    if not cat1_Txt and not cat3_Txt and cat5_Txt and not cat7_Txt:
        Combine="cp Datacard_"+mass+"_cat5.txt Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d  Datacard_"+mass+"_2017.root --run expected"
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        Combinecommand.append(Combine)
        txttoroot.append(getroot)
    if not cat1_Txt and cat3_Txt and not cat5_Txt and not cat7_Txt:
        Combine="cp Datacard_"+mass+"_cat3.txt Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d  Datacard_"+mass+"_2017.root --run expected"
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        Combinecommand.append(Combine)
        txttoroot.append(getroot)
    if cat1_Txt and not cat3_Txt and not cat5_Txt and not cat7_Txt:
        Combine="cp Datacard_"+mass+"_cat1.txt Datacard_"+mass+"_2017.txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+mass+" -d  Datacard_"+mass+"_2017.root --run expected"
        getroot="text2workspace.py Datacard_"+mass+"_2017.txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        Combinecommand.append(Combine)
        txttoroot.append(getroot)
for i in range(0,len(getlimit)):
    print(cd[i])
    print(Combinecommand[i])
    print(txttoroot[i])
    print(getlimit[i])


cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/datacard_M3000
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/datacard_M3000/Datacard_M3000_cat5highpt.txt Datacard_M3000_cat5.txt
text2workspace.py Datacard_M3000_cat5.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat5_M3000 -d Datacard_M3000_cat5.root --run expected    


In [26]:
cat_name = ["cat5highpt","cat5lowpt"]#"M2000","M2600","M2800","M3000"
# cat_name = ["cat7highpt","cat7lowpt"],"M1300"
mass_name = ["M250","M300","M400","M500","M600","M700","M800","M900","M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]
cd=[]
getlimit=[]
Combinecommand=[]
txttoroot=[]
for mass in mass_name:
    datacardpath="/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_"+mass
    cat0_txt= glob.glob(os.path.join(datacardpath, 'Datacard_'+mass+'_'+cat_name[0]+'.txt'))
    cat1_txt= glob.glob(os.path.join(datacardpath, 'Datacard_'+mass+'_'+cat_name[1]+'.txt'))
    # os.chdir("/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_"+mass)
    cdpath="cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2017datacard/datacard_"+mass
    if len(cat0_txt) and len(cat1_txt):
        Combine="combineCards.py  "+cat_name[0].split("high")[0]+"_"+mass+"=Datacard_"+mass+"_"+cat_name[0]+".txt Datacard_"+mass+"_"+cat_name[1]+".txt > Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        # combinecard.append(Combine)
        # getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        cd.append(cdpath)
        getlimit.append(GetLimit)
        txttoroot.append(getroot)
        # subprocess.call(Combine, shell=True, stdout=subprocess.PIPE)
        Combinecommand.append(Combine)
    if len(cat0_txt) and not len(cat1_txt):
        Combine= "cp "+cat0_txt[0]+" Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        cd.append(cdpath)
        Combinecommand.append(Combine)
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        txttoroot.append(getroot)

        # subprocess.run(["cp", cat0_txt, "Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"])
    if not len(cat0_txt) and len(cat1_txt):
        Combine= "cp "+cat1_txt[0]+" Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"
        cd.append(cdpath)
        Combinecommand.append(Combine)
        GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_"+cat_name[0].split("high")[0]+"_"+mass+" -d Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".root --run expected    "
        getlimit.append(GetLimit)
        getroot="text2workspace.py Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt -m 125"
        txttoroot.append(getroot)
        # subprocess.run(["cp", cat1_txt, "Datacard_"+mass+"_"+cat_name[0].split("high")[0]+".txt"])
    if not len(cat0_txt) and not len(cat1_txt):
        print("#no datacard")
for i in range(0,len(getlimit)):
    print(cd[i])
    print(Combinecommand[i])
    print(txttoroot[i])
    print(getlimit[i])

    

#no datacard
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/datacard_M300
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/datacard_M300/Datacard_M300_cat5lowpt.txt Datacard_M300_cat5.txt
text2workspace.py Datacard_M300_cat5.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat5_M300 -d Datacard_M300_cat5.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/datacard_M400
cp /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/datacard_M400/Datacard_M400_cat5lowpt.txt Datacard_M400_cat5.txt
text2workspace.py Datacard_M400_cat5.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_cat5_M400 -d Datacard_M400_cat5.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/datacard_M500
combineCards.py  cat5_M500=Datacard_M500_cat5highpt.txt Datacard_M500_cat5lowpt.txt > Datacard_M500_cat5.txt
text2workspace.py Datacard_M500_c

# combine all categories datacard in each mass point

In [7]:
# mass_name=["M250","M260","M270","M280","M300","M320","M350","M400","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900","M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]
mass_name=["M250","M260","M270","M280","M300","M320","M350","M450","M500","M550","M600","M650","M700","M750","M800","M850","M900","M1000","M1100","M1200","M1300","M1400","M1500","M1600","M1700","M1800","M1900","M2000","M2200","M2400","M2600","M2800","M3000"]
# mass_name=["M550"]
getlimit=[]
Combinecommand=[]
txttoroot=[]
cd = []
l_cat1=[]
l_cat2=[]
l_cat3=[]
l_cat5=[]
l_cat7=[]
l_cat8=[]
for mass in mass_name:
    cdpath="cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_"+mass
    cat1_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_"+mass+"/higgsCombinecombineFHSL_cat1_"+mass+".AsymptoticLimits.mH125.root"
    cat2_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_"+mass+"/higgsCombinecombineFHSL_cat2_"+mass+".AsymptoticLimits.mH125.root"
    cat3_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_"+mass+"/higgsCombinecombineFHSL_cat3_"+mass+".AsymptoticLimits.mH125.root"
    cat5_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_"+mass+"/higgsCombinecombineFHSL_cat5_"+mass+".AsymptoticLimits.mH125.root"
    cat7_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_"+mass+"/higgsCombinecombineFHSL_cat7_"+mass+".AsymptoticLimits.mH125.root"
    cat8_path = "/afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_"+mass+"/higgsCombinecombineFHSL_cat8_"+mass+".AsymptoticLimits.mH125.root"
# 检查文件是否存在，并获取文件大小
    cat1_Txt = os.path.exists(cat1_path) and os.path.getsize(cat1_path) >= 1024
    cat2_Txt = os.path.exists(cat2_path) and os.path.getsize(cat2_path) >= 1024
    cat3_Txt = os.path.exists(cat3_path) and os.path.getsize(cat3_path) >= 1024
    cat5_Txt = os.path.exists(cat5_path) and os.path.getsize(cat5_path) >= 1024
    cat7_Txt = os.path.exists(cat7_path) and os.path.getsize(cat7_path) >= 1024
    cat8_Txt = os.path.exists(cat8_path) and os.path.getsize(cat8_path) >= 1024
    # cat8_Txt = True
    l_cat1.append(int(cat1_Txt)*int(mass.split("M")[1]))
    l_cat2.append(int(cat2_Txt)*int(mass.split("M")[1]))
    l_cat3.append(int(cat3_Txt)*int(mass.split("M")[1]))
    l_cat5.append(int(cat5_Txt)*int(mass.split("M")[1]))
    l_cat7.append(int(cat7_Txt)*int(mass.split("M")[1]))
    l_cat8.append(int(cat8_Txt)*int(mass.split("M")[1]))
    
    # 定义所有的txt文件和对应的cat_Txt
    allcat_txt = ["Datacard_"+mass+"_cat1.txt", "Datacard_"+mass+"_cat2.txt", "Datacard_"+mass+"_cat3.txt", "Datacard_"+mass+"_cat5.txt", "Datacard_"+mass+"_cat7.txt", "Datacard_"+mass+"_cat8.txt"]
    ifcat_txt = [cat1_Txt, cat2_Txt, cat3_Txt, cat5_Txt, cat7_Txt, cat8_Txt]
    exist_txt=[txt for txt, condition in zip(allcat_txt, ifcat_txt) if condition]
    result_txtstr = ' '.join(exist_txt)
    Combine="combineCards.py  Datacard_2016postallcat_"+mass+"="+result_txtstr+" > Datacard_"+mass+"_2016postallcat.txt"
    GetLimit="combine -M AsymptoticLimits -m 125 -n combineFHSL_2016postallcat_"+mass+" -d Datacard_"+mass+"_2016postallcat.root --run expected    "
    getroot="text2workspace.py Datacard_"+mass+"_2016postallcat.txt -m 125"
    cd.append(cdpath)
    getlimit.append(GetLimit)
    txttoroot.append(getroot)
    Combinecommand.append(Combine)
for i in range(0,len(getlimit)):
    print(cd[i])
    print(Combinecommand[i])
    print(txttoroot[i])
    print(getlimit[i])
    



    

cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_M250
combineCards.py  Datacard_2016postallcat_M250=Datacard_M250_cat2.txt Datacard_M250_cat8.txt > Datacard_M250_2016postallcat.txt
text2workspace.py Datacard_M250_2016postallcat.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_2016postallcat_M250 -d Datacard_M250_2016postallcat.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_M260
combineCards.py  Datacard_2016postallcat_M260=Datacard_M260_cat1.txt Datacard_M260_cat2.txt Datacard_M260_cat7.txt Datacard_M260_cat8.txt > Datacard_M260_2016postallcat.txt
text2workspace.py Datacard_M260_2016postallcat.txt -m 125
combine -M AsymptoticLimits -m 125 -n combineFHSL_2016postallcat_M260 -d Datacard_M260_2016postallcat.root --run expected    
cd /afs/cern.ch/user/s/shsong/CMSSW_10_6_20/src/flashggFinalFit/Datacard/2016postdatacard/datacard_M270
combineCards.py  Dat

In [6]:
#pre
print(l_cat1)
print(l_cat2)
print(l_cat3)
print(l_cat5)
print(l_cat7)
print(l_cat8)

[250, 260, 270, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2200, 2400, 2600, 2800, 3000]
[250, 260, 270, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[250, 260, 270, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2200, 2400, 2600, 2800, 3000]
[0, 0, 0, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2200, 2400, 2600, 2800, 3000]
[0, 260, 0, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1100, 0, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2200, 2400, 2600, 0, 0]
[250, 260, 270, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 0, 900, 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [8]:
#post
print(l_cat1)
print(l_cat2)
print(l_cat3)
print(l_cat5)
print(l_cat7)
print(l_cat8)

[0, 260, 270, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 0, 1100, 1200, 1300, 1400, 1500, 0, 1700, 1800, 1900, 2000, 2200, 2400, 2600, 2800, 3000]
[250, 260, 270, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 270, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2200, 2400, 2600, 2800, 3000]
[0, 0, 0, 0, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2200, 2400, 2600, 2800, 3000]
[0, 260, 0, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 1100, 0, 0, 1400, 1500, 0, 1700, 1800, 1900, 2000, 2200, 2400, 2600, 2800, 3000]
[250, 260, 270, 280, 300, 320, 350, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 1000, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
